## Model Training

#### Importing Data and Necessary Libraries



In [31]:
!pip install catboost 



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached catboost-1.2.5-cp311-cp311-win_amd64.whl.metadata (1.2 kB)
  Using cached graphviz-0.20.3-py3-none-any.whl.metadata (12 kB)
   ---------------------------------------- 0.0/101.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/101.1 MB 245.8 kB/s eta 0:06:52
   ---------------------------------------- 0.1/101.1 MB 291.5 kB/s eta 0:05:47
   ---------------------------------------- 0.1/101.1 MB 291.5 kB/s eta 0:05:47
   ---------------------------------------- 0.1/101.1 MB 291.5 kB/s eta 0:05:47
   ---------------------------------------- 0.2/101.1 MB 286.7 kB/s eta 0:05:53
   ---------------------------------------- 0.2/101.1 MB 286.7 kB/s eta 0:05:53
   ----------------------

In [36]:
!pip install xgboost


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached xgboost-2.1.0-py3-none-win_amd64.whl.metadata (2.1 kB)
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.1/124.9 MB 210.1 kB/s eta 0:09:55
   ---------------------------------------- 0.1/124.9 MB 210.1 kB/s eta 0:09:55
   ---------------------------------------- 0.1/124.9 MB 210.1 kB/s eta 0:09:55
   ---------------------------------------- 0.1/12

In [3]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings


### Importing dataset

In [225]:
df = pd.read_csv('data/cleaned_dataset.csv')
data = df.copy()

In [226]:
df.shape

(23469, 24)

In [227]:
df.head()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,VIN,drive,type,paint_color,image_url,description,state,lat,long,posting_date
0,7315233705,https://omaha.craigslist.org/ctd/d/fremont-201...,omaha / council bluffs,https://omaha.craigslist.org,36995,2013.0,ram,2500,good,6 cylinders,...,3C6UR5DL5DG584817,4wd,pickup,black,https://images.craigslist.org/00o0o_bp0dB4eW2i...,2013 Ram 2500 Crew Cab Big Horn Pickup 4D 6 1/...,ia,41.424186,-96.499908,2021-05-01T07:32:52-0500
1,7306636389,https://tricities.craigslist.org/cto/d/bristol...,tri-cities,https://tricities.craigslist.org,10000,2011.0,bmw,550i,good,6 cylinders,...,1FMJU1JT1HEA52352,4wd,sedan,grey,https://images.craigslist.org/00w0w_53YhtQfYp5...,This car is in good condition. Runs and drives...,tn,36.568600,-82.181900,2021-04-14T10:12:48-0400
2,7316498014,https://greenville.craigslist.org/ctd/d/greenv...,greenville / upstate,https://greenville.craigslist.org,34990,2018.0,jeep,grand cherokee trailhawk,good,6 cylinders,...,1C4RJFLG1JC138453,4wd,other,black,https://images.craigslist.org/00202_7MHlVPbgCs...,Carvana is the safer way to buy a car During t...,sc,34.830000,-82.370000,2021-05-03T19:22:00-0400
3,7311169521,https://oklahomacity.craigslist.org/cto/d/hebe...,oklahoma city,https://oklahomacity.craigslist.org,6000,1964.0,chevrolet,shortbed,good,6 cylinders,...,1FMJU1JT1HEA52352,4wd,sedan,white,https://images.craigslist.org/00a0a_4W3LC9j87R...,1964 Chevy short bed step side rolling body. ...,ok,35.510300,-92.039200,2021-04-22T20:24:36-0500
4,7310457441,https://yakima.craigslist.org/ctd/d/kittitas-2...,yakima,https://yakima.craigslist.org,12988,2007.0,toyota,highlander,excellent,6 cylinders,...,1FMJU1JT1HEA52352,4wd,sedan,white,https://images.craigslist.org/00f0f_iKWWuC1aUz...,ＷＩＮＤＹ ＣＨＥＶＲＯＬＥＴ ⭐ No Payments For 90 Days* ⭐...,wa,46.999600,-120.516300,2021-04-21T11:26:09-0700


In [228]:
columns_to_drop = ['url','lat','title_status','long','paint_color','id', 'region_url', 'VIN', 'image_url', 'description', 'state', 'region', 'posting_date']

# Drop columns from df
df = df.drop(columns=columns_to_drop, axis=1)

In [229]:
df

,price,year,manufacturer,model,condition,cylinders,fuel,odometer,transmission,drive,type
0,36995,2013.0,ram,2500,good,6 cylinders,diesel,133863.0,automatic,4wd,pickup
1,10000,2011.0,bmw,550i,good,6 cylinders,gas,112000.0,manual,4wd,sedan
2,34990,2018.0,jeep,grand cherokee trailhawk,good,6 cylinders,gas,37509.0,other,4wd,other
3,6000,1964.0,chevrolet,shortbed,good,6 cylinders,gas,72812.0,manual,4wd,sedan
4,12988,2007.0,toyota,highlander,excellent,6 cylinders,gas,91510.0,automatic,4wd,sedan
...,...,...,...,...,...,...,...,...,...,...,...
23464,2990,2004.0,chevrolet,tahoe ls,good,8 cylinders,gas,254000.0,automatic,4wd,SUV
23465,22500,2002.0,toyota,tacoma xtracab,good,6 cylinders,gas,98368.0,manual,4wd,pickup
23466,9997,2011.0,mercedes-benz,c-class,excellent,6 cylinders,gas,109646.0,automatic,rwd,sedan
23467,16590,2017.0,ford,taurus se sedan 4d,good,6 cylinders,gas,35599.0,other,fwd,sedan


In [230]:
mode = df['cylinders'].mode().iloc[0]
df['cylinders'] = pd.to_numeric(df['cylinders'].str.replace(' cylinders', ''), errors='coerce').fillna(mode)


In [231]:
df

,price,year,manufacturer,model,condition,cylinders,fuel,odometer,transmission,drive,type
0,36995,2013.0,ram,2500,good,6.0,diesel,133863.0,automatic,4wd,pickup
1,10000,2011.0,bmw,550i,good,6.0,gas,112000.0,manual,4wd,sedan
2,34990,2018.0,jeep,grand cherokee trailhawk,good,6.0,gas,37509.0,other,4wd,other
3,6000,1964.0,chevrolet,shortbed,good,6.0,gas,72812.0,manual,4wd,sedan
4,12988,2007.0,toyota,highlander,excellent,6.0,gas,91510.0,automatic,4wd,sedan
...,...,...,...,...,...,...,...,...,...,...,...
23464,2990,2004.0,chevrolet,tahoe ls,good,8.0,gas,254000.0,automatic,4wd,SUV
23465,22500,2002.0,toyota,tacoma xtracab,good,6.0,gas,98368.0,manual,4wd,pickup
23466,9997,2011.0,mercedes-benz,c-class,excellent,6.0,gas,109646.0,automatic,rwd,sedan
23467,16590,2017.0,ford,taurus se sedan 4d,good,6.0,gas,35599.0,other,fwd,sedan


In [232]:
# Replace categories
df['condition'] = df['condition'].replace(['Fair', 'Good', 'Excellent'], ['fair', 'good', 'excellent'])
df['condition'] = np.where(df['condition'].isin(['fair', 'good', 'excellent']), df['condition'], 'good')

# Get unique values in the 'condition' column
unique_conditions = np.unique(df['condition'])

# Define the ordinal encoding mapping
condition_mapping = [list(unique_conditions)]

# Create an OrdinalEncoder object
encoder = OrdinalEncoder(categories=condition_mapping)

# Fit and transform the 'condition' variable
df['condition'] = encoder.fit_transform(df['condition'].values.reshape(-1, 1))

In [233]:
df

,price,year,manufacturer,model,condition,cylinders,fuel,odometer,transmission,drive,type
0,36995,2013.0,ram,2500,2.0,6.0,diesel,133863.0,automatic,4wd,pickup
1,10000,2011.0,bmw,550i,2.0,6.0,gas,112000.0,manual,4wd,sedan
2,34990,2018.0,jeep,grand cherokee trailhawk,2.0,6.0,gas,37509.0,other,4wd,other
3,6000,1964.0,chevrolet,shortbed,2.0,6.0,gas,72812.0,manual,4wd,sedan
4,12988,2007.0,toyota,highlander,0.0,6.0,gas,91510.0,automatic,4wd,sedan
...,...,...,...,...,...,...,...,...,...,...,...
23464,2990,2004.0,chevrolet,tahoe ls,2.0,8.0,gas,254000.0,automatic,4wd,SUV
23465,22500,2002.0,toyota,tacoma xtracab,2.0,6.0,gas,98368.0,manual,4wd,pickup
23466,9997,2011.0,mercedes-benz,c-class,0.0,6.0,gas,109646.0,automatic,rwd,sedan
23467,16590,2017.0,ford,taurus se sedan 4d,2.0,6.0,gas,35599.0,other,fwd,sedan


In [248]:

drive_mapping = {'4wd': 3, 'rwd': 2, 'fwd': 1}
df['drive'] = df['drive'].map(drive_mapping)

In [249]:
df

,price,year,manufacturer,model,condition,cylinders,fuel,odometer,transmission,drive,type
0,36995,2013.0,2500.0,2500.0,2.0,6.0,nan,133863.0,nan,NaN,nan
1,10000,2011.0,nan,nan,2.0,6.0,nan,112000.0,nan,NaN,nan
2,34990,2018.0,nan,nan,2.0,6.0,nan,37509.0,nan,NaN,nan
3,6000,1964.0,nan,nan,2.0,6.0,nan,72812.0,nan,NaN,nan
4,12988,2007.0,nan,nan,0.0,6.0,nan,91510.0,nan,NaN,nan
...,...,...,...,...,...,...,...,...,...,...,...
23464,2990,2004.0,nan,nan,2.0,8.0,nan,254000.0,nan,NaN,nan
23465,22500,2002.0,nan,nan,2.0,6.0,nan,98368.0,nan,NaN,nan
23466,9997,2011.0,nan,nan,0.0,6.0,nan,109646.0,nan,NaN,nan
23467,16590,2017.0,nan,nan,2.0,6.0,nan,35599.0,nan,NaN,nan


### Preparing X and Y Variables

In [250]:
X = df.drop(columns=['price'],axis=1)

In [251]:
X.shape

(23469, 10)

In [252]:
y = df['price']

In [253]:
y

0        36995
1        10000
2        34990
3         6000
4        12988
         ...  
23464     2990
23465    22500
23466     9997
23467    16590
23468    20590
Name: price, Length: 23469, dtype: int64

### Column Transformation

In [254]:
df['year'] = pd.to_numeric(df['year'], errors='coerce')
df['odometer'] = pd.to_numeric(df['odometer'], errors='coerce')
df['condition'] = pd.to_numeric(df['condition'], errors='coerce')
df['drive'] = pd.to_numeric(df['drive'], errors='coerce')

df['manufacturer'] = pd.to_numeric(df['model'], errors='coerce').astype(str)
df['model'] = pd.to_numeric(df['model'], errors='coerce').astype(str)
df['cylinders'] = pd.to_numeric(df['cylinders'], errors='coerce').astype(str)
df['fuel'] = pd.to_numeric(df['fuel'], errors='coerce').astype(str)
df['transmission'] = pd.to_numeric(df['transmission'], errors='coerce').astype(str)
df['type'] = pd.to_numeric(df['type'], errors='coerce').astype(str)


In [255]:
num_features = X.select_dtypes(exclude='object').columns
cat_features = X.select_dtypes(include='object').columns

In [256]:
num_features

Index(['year', 'condition', 'odometer', 'drive'], dtype='object')

In [257]:
cat_features

Index(['manufacturer', 'model', 'cylinders', 'fuel', 'transmission', 'type'], dtype='object')

#### Importing necessary Libraries for Preprocessing and carrying out Preprocessing

In [258]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

In [259]:
numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder(handle_unknown='ignore')

In [260]:
# Create the ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("OneHotEncoder", oh_transformer, cat_features),
        ("StandardScaler", numeric_transformer, num_features),
    ]
)

In [261]:
X = preprocessor.fit_transform(X)

C:\Users\Goodness\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:1051: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\Goodness\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:1056: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
C:\Users\Goodness\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:1076: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [262]:
from sklearn.model_selection import train_test_split
import pandas as pd

# Remove NaN values from the dataset
X.eliminate_zeros()
if isinstance(y, (pd.Series, pd.DataFrame)):
    y.dropna(inplace=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [263]:
import numpy as np

# Assume X is your sparse matrix

# Check for NaN values
if np.isnan(X.data).any():
    # NaN values present, handle them
    print("NaN values found!")
    # Replace NaN values with the mean of the data (or any other suitable value)
    mean_val = np.nanmean(X.data)
    X.data[np.isnan(X.data)] = mean_val
    print(f"NaN values replaced with mean value {mean_val}")
else:
    print("No NaN values found!")

NaN values found!
NaN values replaced with mean value 0.6666666666666661


In [264]:
if np.isnan(X.data).any():
    # NaN values present, handle them
    print("NaN values found!")
else:
    print("No NaN values found!")

No NaN values found!


### Creating an Evaluation Function to give metrics after model training

In [265]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [271]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

models = {
    "Linear Regression": Pipeline([('imputer', SimpleImputer(strategy='mean')), ('lr', LinearRegression())]),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}

model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    
    # Define hyperparameter tuning space
    param_grid = {
    
    }

    # Define cross-validation scheme
    cv = 5

    # Define evaluation metrics
    scoring = ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']

    # Perform hyperparameter tuning with cross-validation
    grid_search = GridSearchCV(model, param_grid, cv=cv, scoring=scoring, refit='r2')
    grid_search.fit(X_train, y_train)

    # Print best hyperparameters and evaluation metrics
    print("Best hyperparameters:", grid_search.best_params_)
    print("Best R2 score:", grid_search.best_score_)
    print("Best MSE:", -grid_search.cv_results_['mean_test_neg_mean_squared_error'][grid_search.best_index_])
    print("Best MAE:", -grid_search.cv_results_['mean_test_neg_mean_absolute_error'][grid_search.best_index_])

    # Train final model with best hyperparameters
    final_model = grid_search.best_estimator_
    final_model.fit(X_train, y_train)

    # Evaluate final model on test set
    y_pred = final_model.predict(X_test)
    print("Final R2 score:", r2_score(y_test, y_pred))
    print("Final MSE:", mean_squared_error(y_test, y_pred))
    print("Final MAE:", mean_absolute_error(y_test, y_pred))
    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(mean_squared_error(y_test, y_pred)))
    print("- Mean Absolute Error: {:.4f}".format(mean_absolute_error(y_test, y_pred)))
    print("- R2 Score: {:.4f}".format(r2_score(y_test, y_pred)))
    r2_list.append(r2_score(y_test, y_pred))
    
    print('='*35)
    print('\n')

Best hyperparameters: {}
Best R2 score: 0.16182980723363535
Best MSE: 303692892.6120405
Best MAE: 9655.76039075715
Final R2 score: 0.21806081782784525
Final MSE: 167094763.55006912
Final MAE: 9316.316517818294
Linear Regression
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 167094763.5501
- Mean Absolute Error: 9316.3165
- R2 Score: 0.2181


Best hyperparameters: {}
Best R2 score: 0.16127600261322012
Best MSE: 303784048.20342594
Best MAE: 9657.06805685098
Final R2 score: 0.21593029350910653
Final MSE: 167550041.23073015
Final MAE: 9320.36665119969
Lasso
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 167550041.2307
- Mean Absolute Error: 9320.3667
- R2 Score: 0.2159


Best hyperparameters: {}
Best R2 score: 0.16216767598668116
Best MSE: 303606946.4155723
Best MAE: 9656.090247342323
Final R2 score: 0.217062862706491
Final MSE: 167308019.36182296
Final MAE: 9317.156749784435
Ridge
-----------------